In [1]:
import pandas as pd

## EDA questions:

* What are the most common diagnoses
    * Which disease do we want to investigate?
* What are the most common lab tests run for patients with/ wo CHF
    * Which labs should we use as features?
* What diagnoses are associated wtih CHF (co-morbidity)?
    * Can we use certain diagnosis as features? (coronary artery disease, high blood pressure, diabetes or obesity)
* How many times did patients visit the hospital before their first CHF diagnosis?
    * Justify the need for model

In [41]:
diag = pd.read_csv('./data/hosp/diagnoses_icd.csv.gz', compression='gzip')
diag_d = pd.read_csv('./data/hosp/d_icd_diagnoses.csv.gz', compression='gzip')

In [42]:
# Merge diagnosis data with icd definition
diag = diag.merge(diag_d, how='left', on='icd_code')
diag.long_title.value_counts().head(20)

Unspecified essential hypertension                                                                            81275
Other and unspecified hyperlipidemia                                                                          57076
Essential (primary) hypertension                                                                              53652
Hyperlipidemia, unspecified                                                                                   50353
Acute kidney failure, unspecified                                                                             44091
Personal history of nicotine dependence                                                                       40097
Esophageal reflux                                                                                             40062
Gastro-esophageal reflux disease without esophagitis                                                          35044
Diabetes mellitus without mention of complication, type II or unspecifie

In [60]:
# diag[diag.long_title.str.lower().str.contains('congestive') & diag.long_title.str.lower().str.contains('heart') & diag.long_title.str.lower().str.contains('failure')].icd_code.unique()
diag[diag.long_title.str.lower().str.contains('celiac disease').fillna(False)].long_title.unique()

array(['Celiac disease'], dtype=object)

In [67]:
# Create target table that show whether a patient has had CHF diagnosis or not
# diag['has_CHF']=diag.long_title.str.lower().str.contains('congestive') & diag.long_title.str.lower().str.contains('heart') & diag.long_title.str.lower().str.contains('failure')
# diag.has_CHF.replace({False:0, True:1}, inplace=True)
# subj_hasCHF = diag.groupby('subject_id').sum()['has_CHF'].agg(lambda x: 1 if x>0 else 0).reset_index()


# Create target table that show whether a patient has had celiac disease or not
diag['has_celiac']=diag.long_title.str.lower().str.contains('celiac disease')
diag.has_celiac.replace({False:0, True:1}, inplace=True)
subj_hasCeliac = diag.groupby('subject_id').sum()['has_celiac'].agg(lambda x: 1 if x>0 else 0).reset_index()
adm_hasCeliac = diag.groupby('hadm_id').sum()['has_celiac'].agg(lambda x: 1 if x>0 else 0).reset_index()

In [68]:
subj_hasCeliac.mean()
adm_hasCeliac.mean()
# diag.has_celiac.mean()

hadm_id       2.500397e+07
has_celiac    2.913781e-03
dtype: float64

Celiac Disease: 0.3% of admissions


### ICU
* how many patients with CHF were in the ICU?
* how many patients were in the ICU vs not
* how many visits were in the ICU vs not
* what are the ICU events that are most commonly done?
* How many times is BP (and other vitals/labs) taken for each ICU stay/patient?
* Are these patients found in the diagnosis table?
* What are most common labs for ICU patients?


In [2]:
# import chartevents

chunksize = 3*(10 ** 6)
counter=0
chart_events = []
for chunk in pd.read_csv('./data/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
    chart_events.append(chunk)
    counter+=1
    print(f'chunk {counter} processed')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 1 processed
chunk 2 processed
chunk 3 processed
chunk 4 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 5 processed
chunk 6 processed
chunk 7 processed
chunk 8 processed
chunk 9 processed
chunk 10 processed
chunk 11 processed


KeyboardInterrupt: 

In [ ]:
chart_event_final = pd.concat(chart_events)
# chart_events[0]

In [3]:
d_items = pd.read_csv('./data/icu/d_items.csv.gz', compression='gzip')

In [36]:
d_routine_vitals = d_items[(d_items.linksto =='chartevents')& (d_items.category=='Routine Vital Signs')]
# d_items[(d_items['label'].str.contains('Heart'))]

In [17]:
# d_routine_vitals
# d_items

In [18]:
# chart1_vitals = chart_events[0].merge(d_routine_vitals, how='right', on='itemid')
# chart1_vitals[-chart1_vitals.valuenum.isna()]

In [5]:
# most commonly done chart events
common_events_icu = chart_events[0].groupby(['subject_id','itemid']).count().reset_index().itemid.value_counts().head(100).to_frame().reset_index().merge(d_items, left_on='index', right_on='itemid', how='left')

In [8]:
common_items = common_events_icu[common_events_icu.category != 'Alarms'].head(50).itemid_y.tolist()

In [9]:
chart_1 = chart_events[0]

### Median number of times each common event occured per patient visit

In [38]:
test = chart_1[chart_1.itemid.isin(common_items)].merge(d_items, how='left', on='itemid')
test.groupby(['hadm_id','itemid']).count().reset_index().groupby('itemid').median()[['subject_id']].reset_index().merge(d_items, on='itemid', how='left')

,itemid,subject_id,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220045,51.0,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
1,220179,35.0,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
2,220180,35.0,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
3,220181,36.0,Non Invasive Blood Pressure mean,NBPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
4,220210,50.0,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
5,220228,4.0,Hemoglobin,Hemoglobin,chartevents,Labs,g/dl,Numeric,NaN,NaN
6,220277,50.0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
7,220545,4.0,Hematocrit (serum),Hematocrit (serum),chartevents,Labs,None,Numeric,NaN,NaN
8,220546,3.0,WBC,WBC,chartevents,Labs,None,Numeric,NaN,NaN
9,220602,3.0,Chloride (serum),Chloride (serum),chartevents,Labs,None,Numeric,NaN,NaN


### Are patients in ICU found in diagnosis

In [48]:
test_admits = chart_1.groupby('hadm_id').count().reset_index()[['hadm_id']]
test_admits.merge(diag, how='left', on='hadm_id')
# test_admits

# yes all patients in ICU can be found in diagnosis_ICD table. Keep in mind, multiple diagnosis per visit.
# Patients can have visits outside ICU.

,hadm_id,subject_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title
0,20005763,10628534.0,4.0,E7800,10.0,10.0,"Pure hypercholesterolemia, unspecified"
1,20005763,10628534.0,9.0,R008,10.0,10.0,Other abnormalities of heart beat
2,20005763,10628534.0,6.0,I2510,10.0,10.0,Atherosclerotic heart disease of native corona...
3,20005763,10628534.0,3.0,I10,10.0,10.0,Essential (primary) hypertension
4,20005763,10628534.0,1.0,I7102,10.0,10.0,Dissection of abdominal aorta
...,...,...,...,...,...,...,...
32075,29994501,11031780.0,12.0,53081,9.0,9.0,Esophageal reflux
32076,29994501,11031780.0,7.0,2724,9.0,9.0,Other and unspecified hyperlipidemia
32077,29994501,11031780.0,11.0,496,9.0,9.0,"Chronic airway obstruction, not elsewhere clas..."
32078,29994501,11031780.0,9.0,51889,9.0,9.0,"Other diseases of lung, not elsewhere classified"


In [35]:
d_items[d_items.itemid==220045]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
1,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN


In [21]:
# chart_events[0].groupby(['subject_id','itemid']).count().reset_index().subject_id.nunique()
chart_merged = chart_events[0].merge(d_items, how='left', on='itemid')

# chart_merged.groupby(['subject_id','itemid']).count().reset_index()

In [23]:
chart_merged[chart_merged.itemid==220277]

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
67,10003700,28623837,35053963,2165-04-24 05:31:00,2165-04-24 05:37:00,220277,100,100.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
92,10003700,28623837,35053963,2165-04-24 06:00:00,2165-04-24 05:42:00,220277,100,100.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
120,10003700,28623837,35053963,2165-04-24 06:09:00,2165-04-24 06:09:00,220277,100,100.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
127,10003700,28623837,35053963,2165-04-24 07:00:00,2165-04-24 07:37:00,220277,100,100.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
177,10003700,28623837,35053963,2165-04-24 08:00:00,2165-04-24 08:19:00,220277,100,100.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999900,12426769,26860378,30486026,2164-04-27 20:00:00,2164-04-27 20:51:00,220277,98,98.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
2999943,12426769,26860378,30486026,2164-04-27 21:00:00,2164-04-27 23:46:00,220277,100,100.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
2999970,12426769,26860378,30486026,2164-04-27 22:00:00,2164-04-27 23:46:00,220277,99,99.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
2999981,12426769,26860378,30486026,2164-04-27 23:00:00,2164-04-27 23:46:00,220277,100,100.0,%,0,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN


In [72]:
d_hcpc = pd.read_csv('./data/mimic-iv-0.4/hosp/d_hcpcs.csv.gz', compression='gzip')

In [73]:
d_hcpc[d_hcpc.long_description.fillna('0').str.lower().str.contains('history')]

,code,category,long_description,short_description
97,G9,NaN,Monitored anesthesia care for patient who has ...,Mac for at risk patient
2100,G0245,NaN,Initial physician evaluation and management of...,Initial foot exam pt lops
2101,G0246,NaN,Follow-up physician evaluation and management ...,Followup eval of foot pt lop
2267,G0505,NaN,Cognition and functional assessment using stan...,Cog/func assessmentoutpt
2426,G8560,NaN,Patient has a history of active drainage from ...,Pt hx act drain prev 90 days
2427,G8561,NaN,Patient is not eligible for the referral for o...,Pt inelig for ref oto eval
2428,G8562,NaN,Patient does not have a history of active drai...,Pt no hx act drain 90 d
2619,G8861,NaN,"Within the past 2 years, central dual-energy x...",Dxa ordered for osteo
2979,G9277,NaN,Documentation that the patient is on daily asp...,Doc daily aspirin or contra
2984,G9282,NaN,Documentation of medical reason(s) for not rep...,Doc medrsn no histo type


## Processing for model 

In [69]:
diag[diag.long_title.str.lower().str.contains('celiac disease').fillna(False)].long_title.unique()

array(['Celiac disease'], dtype=object)

In [ ]:
# df.loc[df.reset_index().groupby(['F_Type'])['to_date'].idxmax()]